In [ ]:
# display full with
display("text/html", "<style>.container { width:100% !important; }</style>")

In [ ]:
include("../Flyonic.jl");
using .Flyonic;

using Rotations;
using LoopThrottle;
using LinearAlgebra

In [ ]:
vis = create_visualization();
create_Crazyflie("Crazyflie", actuators = true);
set_Crazyflie_actuators("Crazyflie", [0.0; 0.0; 0.0; 0.0])
create_sphere("sphere"; radius=0.02);

In [ ]:
function figure_8_position(t, x_W, final_time)
    radius = 0.5
    max_speed = 0.2
    ramp_speed = 1.0 # how long in % of the total it should take to reach the max speed
    
    if t < ramp_speed*final_time  
        speed = max_speed * t/(ramp_speed*final_time)
    else
        speed = max_speed
    end
    
    if (mod(t*speed, 4.0*pi) > 2.0*pi)
        p_des_W = [cos(t*speed)*radius-radius,sin(t*speed)*radius,0.5];
    else
        p_des_W = [-(cos(t*speed)*radius-radius),sin(t*speed)*radius,0.5];
    end
    
     v_des_W = (p_des_W - x_W)*4.0
    
    set_transform("sphere", p_des_W);
    
    return p_des_W, v_des_W
end;

In [ ]:
function get_desired_point(x_W, x_W_desired)
    
    
    acceptance_radius = 0.1
    next_point_distance = 0.3
     
    if norm(x_W .- x_W_desired) < acceptance_radius
        direction = (rand(3) .- 0.5) # get random direction
        direction[3] = abs(direction[3])
        
        direction = (direction/norm(direction)) # normalize
        direction = direction * next_point_distance # distance
        
        x_W_desired = x_W .+ direction # update next point
    end
    
    set_transform("sphere", x_W_desired);
    
    return x_W_desired
end;

In [ ]:
function init_PID(KP::Real, KI::Real, iLimit::Real, KD::Real, KFF::Real)

    integ_Error = 0.0;
    prev_Error = 0.0;

    function update_PID(Desired::Real, Actual::Real, DT::Real)

        # Error
        Error = Desired - Actual;

        # P 
        outP = KP * Error;
    
        # I 
        integ_Error += Error * DT;
        integ_Error = clamp(integ_Error, -iLimit, iLimit)
        outI = KI * integ_Error;

        # D
        deriv_error = (Error - prev_Error) / DT;
        # TODO: filter https://github.com/bitcraze/crazyflie-firmware/blob/2748522037cfaa19186d203652da9c723138d0ca/src/modules/src/pid.c#L104
        outD = KD * deriv_error;
        prev_Error = Error;

        # FF
        outFF = KFF * Desired;

        # TODO: filter complete output instead of only D component to compensate for increased noise from increased barometer influence
        # lpf2pApply(&pid->dFilter, output);
        output = outP + outI + outD + outFF;
        # TODO: output = constrain(output, -outputLimit, outputLimit);

        return output;
    end

    return update_PID
end


In [ ]:
function init_crazyflie_controller2(velocity_limit)

    vel_x_PID = init_PID(1.0, 0.0, 0.0, 0.0, 0.0);
    vel_y_PID = init_PID(1.0, 0.0, 0.0, 0.0, 0.0);
    vel_z_PID = init_PID(1.0, 0.0, 0.0, 0.0, 0.0);
    
   

    function positionController(actual_pos_x_W::Real, actual_pos_y_W::Real, actual_pos_z_W::Real,
                                desired_pos_x_W::Real, desired_pos_y_W::Real, desired_pos_z_W::Real,
                                DT::Real)

        desired_vel_x_W = vel_x_PID(desired_pos_x_W, actual_pos_x_W, DT);
        desired_vel_y_W = vel_y_PID(desired_pos_y_W, actual_pos_y_W, DT);
        desired_vel_z_W = vel_z_PID(desired_pos_z_W, actual_pos_z_W, DT);

        return desired_vel_x_W, desired_vel_y_W, desired_vel_z_W;
    end


    thrustMin = 2000.0;#20000.0; # Minimum thrust value to output
    thrustScale = 1000.0;
    thrustBase = 5000.0; # approximate throttle needed when in perfect hover. More weight/older battery can use a higher value
    pLimit = 20.0; # PID_VEL_PITCH_MAX
    rLimit = 20.0; # PID_VEL_ROLL_MAX

    thrust_PID = init_PID(25.0, 15.0, 1_000.0, 0.0, 0.0);
    eulerPitch_PID = init_PID(25.0, 1.0, 1_000.0, 0.0, 0.0); # Vel_X
    eulerRoll_PID = init_PID(25.0, 1.0, 1_000.0, 0.0, 0.0); # Vel_Y
    

    function velocityController(actual_vel_x_B::Real, actual_vel_y_B::Real, actual_vel_z_B::Real, 
                                desired_vel_x_B::Real, desired_vel_y_B::Real, desired_vel_z_B::Real, 
                                DT::Real)
        
        desired_vel_x_B = clamp(desired_vel_x_B, -velocity_limit, velocity_limit);
        desired_vel_y_B = clamp(desired_vel_y_B, -velocity_limit, velocity_limit);
        desired_vel_z_B = clamp(desired_vel_z_B, -velocity_limit, velocity_limit);

        # Roll and Pitch
        eulerPitchDesired = eulerPitch_PID(desired_vel_x_B, actual_vel_x_B, DT);
        eulerRollDesired = -eulerRoll_PID(desired_vel_y_B, actual_vel_y_B, DT);
        eulerPitchDesired = clamp(eulerPitchDesired, -pLimit, pLimit);
        eulerRollDesired  = clamp(eulerRollDesired,  -rLimit, rLimit);

        # Thrust
        thrustRaw = thrust_PID(desired_vel_z_B, actual_vel_z_B, DT);
        # Scale the thrust and add feed forward term
        thrust = thrustRaw * thrustScale + thrustBase;
        # Limit Thrust to UINT16
        thrust = clamp(thrust, thrustMin, 65535.0);

        return thrust, eulerRollDesired, eulerPitchDesired
    end 


    # params from https://github.com/bitcraze/crazyflie-firmware/blob/2748522037cfaa19186d203652da9c723138d0ca/src/platform/interface/platform_defaults_cf2.h#L64
    roll_PID = init_PID(6.0, 3.0, 20.0, 0.0, 0.0);
    pitch_PID = init_PID(6.0, 3.0, 20.0, 0.0, 0.0);
    yaw_PID = init_PID(6.0, 1.0, 360.0, 0.35, 0.0);

    # https://github.com/bitcraze/crazyflie-firmware/blob/2748522037cfaa19186d203652da9c723138d0ca/src/modules/src/attitude_pid_controller.c#L156
    function attitudeControllerCorrectAttitudePID(  eulerRollActual::Real, eulerPitchActual::Real, eulerYawActual::Real,
                                                    eulerRollDesired::Real, eulerPitchDesired::Real, eulerYawDesired::Real,
                                                    DT::Real)

        rollRateDesired  = roll_PID(eulerRollDesired, eulerRollActual, DT);
        pitchRateDesired = pitch_PID(eulerPitchDesired, eulerPitchActual, DT);
    
        # Update PID for yaw axis
        yawError = eulerYawDesired - eulerYawActual;
        if (yawError > 180.0)
            yawError -= 360.0;
        elseif (yawError < -180.0)
            yawError += 360.0;
        end

        eulerYawDesired = yawError + eulerYawActual;
        yawRateDesired = yaw_PID(eulerYawDesired, eulerYawActual, DT);
        yawRateDesired = clamp(yawRateDesired, -20.0, 20.0)

        return rollRateDesired, pitchRateDesired, yawRateDesired
    end


    # params from https://github.com/bitcraze/crazyflie-firmware/blob/2748522037cfaa19186d203652da9c723138d0ca/src/platform/interface/platform_defaults_cf2.h#L45
    rollRate_PID = init_PID(250.0, 500.0, 33.3, 2.5, 0.0);
    pitchRate_PID = init_PID(250.0, 500.0, 33.3, 2.5, 0.0);
    yawRate_PID = init_PID(120.0, 16.7, 166.7, 0.0, 0.0);

    # https://github.com/bitcraze/crazyflie-firmware/blob/2748522037cfaa19186d203652da9c723138d0ca/src/modules/src/attitude_pid_controller.c#L141
    function attitudeControllerCorrectRatePID(
                rollRateActual::Real, pitchRateActual::Real, yawRateActual::Real,
                rollRateDesired::Real, pitchRateDesired::Real, yawRateDesired::Real,
                DT::Real)

                rollOutput  = rollRate_PID(rollRateDesired, rollRateActual, DT);
                pitchOutput = pitchRate_PID(pitchRateDesired, pitchRateActual, DT);
                yawOutput   = yawRate_PID(yawRateDesired, yawRateActual, DT);

                return rollOutput, pitchOutput, yawOutput
    end


    #  https://github.com/bitcraze/crazyflie-firmware/blob/8cbdceeb4ceab868729776b8b6ba6aa48bcc04d5/src/modules/src/power_distribution_quadrotor.c#L79
    function powerDistributionLegacy(roll, pitch, yaw, thrust)
        roll = roll / 2.0;
        pitch = pitch / 2.0;
    
        pitch = -pitch; # Pitch is defined with LEFT hand rule https://www.bitcraze.io/documentation/system/platform/cf2-coordinate-system/
    
        motor_1 = thrust - roll + pitch + yaw;
        motor_2 = thrust - roll - pitch - yaw;
        motor_3 = thrust + roll - pitch + yaw;
        motor_4 = thrust + roll + pitch - yaw;

        return motor_1, motor_2, motor_3, motor_4
    end

    # Position controller
    function crazyflie_PID_controller(  R_W::Matrix,
                                        ω_B::Vector,
                                        v_B,
                                        x_W,
                                        desired_pos_W,
                                        DT::Real)
        
        
        actual_pos_x_W = x_W[1]
        actual_pos_y_W = x_W[2]
        actual_pos_z_W = x_W[3]       
        
        
        desired_pos_x_W = desired_pos_W[1];
        desired_pos_y_W = desired_pos_W[2];
        desired_pos_z_W = desired_pos_W[3];


        # Position PID
        desired_vel_x_W, desired_vel_y_W, desired_vel_z_W = positionController( actual_pos_x_W, actual_pos_y_W, actual_pos_z_W,
                                                                                desired_pos_x_W, desired_pos_y_W, desired_pos_z_W,
                                                                                DT);
        
        desired_v_W = [desired_vel_x_W; desired_vel_y_W; desired_vel_z_W]
        #println(desired_v_W)
        desired_v_B = transpose(R_W)*desired_v_W;

        desired_vel_x_B = desired_v_B[1]
        desired_vel_y_B = desired_v_B[2]
        desired_vel_z_B = desired_v_B[3]

        actual_vel_x_B = v_B[1]
        actual_vel_y_B = v_B[2]
        actual_vel_z_B = v_B[3]
        
        # Velocity PID
        thrustOutput, eulerRollDesired, eulerPitchDesired = velocityController( actual_vel_x_B, actual_vel_y_B, actual_vel_z_B, 
                                                                                desired_vel_x_B, desired_vel_y_B, desired_vel_z_B,
                                                                                DT);
        

        eulerRollActual  = RotZYX(R_W).theta3 * 180.0 / pi
        eulerPitchActual = RotZYX(R_W).theta2 * 180.0 / pi
        eulerYawActual   = RotZYX(R_W).theta1 * 180.0 / pi

        eulerYawDesired = 0.0;


        # Altitude PID
        rollRateDesired, pitchRateDesired, yawRateDesired = attitudeControllerCorrectAttitudePID(   eulerRollActual, eulerPitchActual, eulerYawActual,
                                                                                                    eulerRollDesired, eulerPitchDesired, eulerYawDesired,
                                                                                                    DT);

        rollRateActual  = ω_B[1]
        pitchRateActual = ω_B[2]
        yawRateActual   = ω_B[3]

        # Altitude Rate PID
        rollOutput, pitchOutput, yawOutput = attitudeControllerCorrectRatePID(  rollRateActual, pitchRateActual, yawRateActual,
                                                                                rollRateDesired, pitchRateDesired, yawRateDesired, 
                                                                                DT);

        
        # Power distribution 
        motor_1, motor_2, motor_3, motor_4 = powerDistributionLegacy(rollOutput, pitchOutput, yawOutput, thrustOutput);


        return motor_1, motor_2, motor_3, motor_4
    end


    return crazyflie_PID_controller
end


In [ ]:
function run_PID()
    
    Δt = 0.001
    
    velocity_limit = 2;# m/s
    crazyflie_PID_controller = init_crazyflie_controller2(velocity_limit)
    

    x_W = [0.0; 0.0; 0.0]
    R_W = UnitQuaternion(1.0, 0.0, 0.0, 0.0)
    ω_B = [0.0; 0.0; 0.0]
    v_B = [0.0; 0.0; 0.0]
    desired_pos_W = [0.0; 0.0; 0.0]
    t = 0.0
    
    final_time = 5.0
    
    # Loop throttle for human visualisation
    @throttle t while t <= final_time
        
        # desired_pos_W, _ = figure_8_position(t, x_W, 5.0);
        
        desired_pos_W = get_desired_point(x_W, desired_pos_W)

        

        motor_1, motor_2, motor_3, motor_4 = crazyflie_PID_controller( Matrix(R_W), 
                                                                       ω_B,
                                                                       v_B,
                                                                       x_W,
                                                                       desired_pos_W,
                                                                       Δt)


        action = [motor_1; motor_2; motor_3; motor_4]
        
        # caluclate aerodynamic forces
        torque_B, force_B = crazyflie_model(action);
        
        # noise
        #ω_B += ω_B.*(rand(3).-0.5).*2.0*0.01 # +- 10%
        #v_B += v_B.*(rand(3).-0.5).*2.0*0.01 # +- 10%

        # set the next forces for the rigid body simulation
        x_W, v_B, _, R_W, ω_B, _, t = rigid_body_simple(torque_B, force_B, x_W, v_B, R_W, ω_B, t, Δt, crazyflie_param)
        
        #println(typeof(v_B))
        #println(typeof(x_W))
                
        set_Crazyflie_actuators("Crazyflie", action ./ 10000.0)
        set_transform("Crazyflie", x_W, QuatRotation(R_W));

        
        
    end max_rate = 1.0
end;

In [ ]:
run_PID()